In [ ]:
import pandas as pd

In [33]:
df  = pd.read_csv("microclimate-sensors-data.csv")
df.shape

(398946, 16)

In [23]:
missing_entries = df.isnull().sum()
for col,count in missing_entries.items():
    print(f"{col}: {count} missing entries")

Device_id: 0 missing entries
Time: 0 missing entries
SensorLocation: 6143 missing entries
LatLong: 11483 missing entries
MinimumWindDirection: 40111 missing entries
AverageWindDirection: 504 missing entries
MaximumWindDirection: 40269 missing entries
MinimumWindSpeed: 40269 missing entries
AverageWindSpeed: 504 missing entries
GustWindSpeed: 40269 missing entries
AirTemperature: 504 missing entries
RelativeHumidity: 504 missing entries
AtmosphericPressure: 504 missing entries
PM25: 19039 missing entries
PM10: 19039 missing entries
Noise: 19039 missing entries


In [24]:
df_new = df.copy()

In [25]:
locToLatlongMap = (
    df[df['LatLong'].notnull()]
    .groupby('SensorLocation')['LatLong']
    .first()
    .to_dict()
)

print(locToLatlongMap)

{'1 Treasury Place': '-37.8128595, 144.9745395', '101 Collins St L11 Rooftop': '-37.814604, 144.9702991', 'Batman Park': '-37.8221828, 144.9562225', 'Birrarung Marr Park - Pole 1131': '-37.8185931, 144.9716404', 'CH1 rooftop': '-37.8140348, 144.96728', 'Enterprize Park - Pole ID: COM1667': '-37.8204083, 144.9591192', 'Royal Park Asset ID: COM2707': '-37.7956167, 144.9519007', "SkyFarm (Jeff's Shed). Rooftop - Melbourne Conference & Exhibition Centre (MCEC)": '-37.8223306, 144.9521696', 'Swanston St - Tram Stop 13 adjacent Federation Sq & Flinders St Station': '-37.8184515, 144.9678474', 'Tram Stop 7B - Melbourne Tennis Centre Precinct - Rod Laver Arena': '-37.8194993, 144.9787211', 'Tram Stop 7C - Melbourne Tennis Centre Precinct - Rod Laver Arena': '-37.8222341, 144.9829409'}


In [ ]:
rows_to_drop = df_new['SensorLocation'].isnull() & df_new['LatLong'].isnull()

df_cleaned = df_new[~rows_to_drop].copy()

In [29]:
print(df_cleaned['SensorLocation'].isnull().sum())

print(df_cleaned['LatLong'].isnull().sum())


0
5340


In [30]:
df_cleaned['LatLong'] = df_cleaned.apply(
    lambda row:locToLatlongMap[row['SensorLocation']] if pd.isnull(row['LatLong']) else row['LatLong'],
    axis=1
)
print(df_cleaned['LatLong'].isnull().sum())

0


In [34]:
missing_entries = df_cleaned.isnull().sum()
for col,count in missing_entries.items():
    print(f"{col}: {count} missing entries")
df_cleaned.shape

Device_id: 0 missing entries
Time: 0 missing entries
SensorLocation: 0 missing entries
LatLong: 0 missing entries
MinimumWindDirection: 39852 missing entries
AverageWindDirection: 352 missing entries
MaximumWindDirection: 40010 missing entries
MinimumWindSpeed: 40010 missing entries
AverageWindSpeed: 352 missing entries
GustWindSpeed: 40010 missing entries
AirTemperature: 352 missing entries
RelativeHumidity: 352 missing entries
AtmosphericPressure: 352 missing entries
PM25: 18839 missing entries
PM10: 18839 missing entries
Noise: 18839 missing entries


(392803, 16)